In [1]:
import cv2
import numpy as np

In [2]:
video = cv2.VideoCapture(r"C:\Users\hp\Videos\Screen Recordings\arrow.mp4")


In [3]:

# Check if video opened successfully
if not video.isOpened():
    print("Error: Could not open video.")
    exit()

# Read the first frame
ret, prev_frame = video.read()
if not ret:
    print("Error: Unable to read video frame.")
    video.release()
    exit()

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

# Parameters for feature detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7
)

# Detect good features to track in the first frame
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

# Ensure features are detected
if prev_corners is None:
    print("Error: No features detected!")
    video.release()
    exit()

# Create a mask image for drawing
mask = np.zeros_like(prev_frame)

while True:
    # Read the next frame
    ret, current_frame = video.read()
    if not ret:
        break  # Exit if video ends

    # Convert to grayscale
    current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    current_corners, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, current_gray, prev_corners, None, **lk_params)

    # Ensure valid points exist
    if current_corners is None or status is None:
        print("Warning: Optical flow tracking failed.")
        break

    # Select good points
    good_new = current_corners[status.flatten() == 1]
    good_old = prev_corners[status.flatten() == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        current_frame = cv2.circle(current_frame, (int(a), int(b)), 5, (0, 0, 255), -1)

    # Overlay the mask on the frame
    output = cv2.add(current_frame, mask)

    # Show the output
    cv2.imshow("Optical Flow", output)

    # Break loop on 'q' key
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update previous frame and points
    prev_gray = current_gray.copy()
    prev_corners = good_new.reshape(-1, 1, 2) if len(good_new) > 0 else None

    # If no more good points are left, re-detect them
    if prev_corners is None or len(prev_corners) == 0:
        prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
        if prev_corners is None:
            print("No new features detected, exiting...")
            break

# Release video and close windows
video.release()
cv2.destroyAllWindows()
